In [1]:
import pandas as pd
import numpy as np
from sklearn.svm import LinearSVC
from sklearn import metrics
from sklearn import datasets
from numpy.linalg import norm, inv
from sklearn import model_selection

In [2]:
data = datasets.load_iris()
X = data.data
y = data.target

# make this a binary case and easier and only use first 2 classes and first 2 features
X = X[y != 2, :2]

y = data.target
y = y[y != 2]

# convert it to -1 and 1 cases for my own method implementation
y[y==0] = -1

# normalize them to have them in the same scale
X = (X - np.mean(X, axis=0))/np.std(X, axis=0)

# split our train-test set
xTrain, xTest, yTrain, yTest = model_selection.train_test_split(X, y, test_size=0.2, random_state=0)

# Sklearn Method

In [3]:
# sklearn implementation
svm = LinearSVC(dual = False).fit(xTrain, yTrain)
yPred = svm.predict(xTest)
accuracy = metrics.accuracy_score(yTest, yPred)
accuracy

1.0

It has an accuracy of 100%

# My Method

In [4]:
# find D as I explain in my hw pdf
D = np.zeros((xTrain.shape[0], xTrain.shape[0]))
for i in range(D.shape[0]):
    for j in range(D.shape[1]): 
        D[i,j] = yTrain[i]*yTrain[j]*xTrain[i].T@xTrain[j]

f = lambda x: 1/2*x.T@D@x - x.T*np.ones(x.shape[0])
gradf = lambda x: D@x-np.ones((x.shape[0]))
hessf = lambda x: D

In [5]:
# regular term, align with the sklearn linear svc default slack value of 1
C = 1

eq_A = yTrain.T.reshape(1,-1) #dimension: 1*n
eq_b = np.zeros((1,1)) #dimension: 1

# inequality constraint
ineq_f = lambda x: np.concatenate((x-C*np.ones(x.shape[0]), -x), axis=0) #dimension: 2*n
ineq_df = lambda x: np.concatenate((np.diag(np.ones(x.shape[0])), np.diag(-np.ones(x.shape[0]))),axis=0) #dimension: 2n*n

In [6]:
# some hyper parameters
t = 4
tol = 1e-8
maxIter = 100 # max iterations
i = 0 # number of iteraitons

# initial iterior point
x_k = 0.01*np.ones(xTrain.shape[0]) #alpha
l_k = np.ones(xTrain.shape[0]*2) #lambda
n_k = 1 #nu

In [7]:
# Define residual function
r_1 = lambda x,l,n: gradf(x) + ineq_df(x).T@l + (eq_A.T*n).reshape(-1);
r_2 = lambda x,l,n: -np.diag(l)@ineq_f(x) - np.ones(l.shape[0])/t;
r_3 = lambda x,l,n: (eq_A@x - eq_b).reshape(-1);
res = lambda x,l,n: np.concatenate((r_1(x,l,n),r_2(x,l,n),r_3(x,l,n)),axis=0)

In [8]:
def lineSearch(dX,dL,dN): 
    maxIter = 20
    alpha = 1
    beta = 0.7
    i = 0 #number of iterations
    while(i < maxIter):
        if ((max(ineq_f(x_k + alpha*dX)) <= 0) and (norm(res(x_k+alpha*dX, l_k+alpha*dL, n_k+alpha*dN)) <= (1 - 0.1*alpha)*norm(res_k))):
            break
        else:      
            alpha *= beta;
            i += 1;
    return alpha

In [9]:
while (i < maxIter): 
    # Compute residual at current iteration
    res_k = res(x_k, l_k, n_k) 
    if(norm(res_k) < tol): 
        break 
    first_row = np.concatenate((hessf(x_k),ineq_df(x_k).T,eq_A.T),axis=1)
    second_row = np.concatenate((-np.diag(l_k)@ineq_df(x_k), -np.diag(ineq_f(x_k)), np.zeros((l_k.shape[0], 1))),axis=1)
    third_row = np.concatenate((eq_A,np.zeros((1, l_k.shape[0])), np.zeros((1, 1))),axis=1)
    # Hessian matrix of linear func is the zero matrix
    dY = -inv(np.concatenate((first_row,second_row,third_row),axis=0))@res_k;
    dX = dY[0:x_k.shape[0]] 
    dL = dY[x_k.shape[0]:x_k.shape[0]*3]
    dN = dY[x_k.shape[0]*3:x_k.shape[0]*3 + 1]
    alpha = lineSearch(dX,dL,dN)
    x_k = x_k+alpha*dX
    l_k = l_k+alpha*dL
    n_k = n_k+alpha*dN
    i += 1
    

In [10]:
# recover W using x_k
W = np.zeros(2)
W[0] = sum(x_k*yTrain*xTrain[:,0])
W[1] = sum(x_k*yTrain*xTrain[:,1])
W

array([ 6.47000886, -6.53086837])

In [11]:
# calculate bias for W.T@x+b
b = np.mean(yTrain-xTrain@W.T)
b

-0.11921465233075304

In [12]:
# predict class y for the xTest using W and bias
yPred = np.zeros(xTest.shape[0])
z = xTest@W.T+b
yPred[z < 0] = -1
yPred[z > 0] = 1

In [13]:
# see results
accuracy = metrics.accuracy_score(yTest, yPred)
accuracy

1.0

Both my svm method and the scikit learn svm method have the same accuracy of 100%!!!